# Bae de Operaciones
### Funciones de filtro para operaciones

In [21]:
import re
import pandas as pd 
from openpyxl import load_workbook
import numpy as np

In [22]:
LUGARES_CATALOGADOS= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

UNIDADES_MUNICIPIOS = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA",
    "RG4": "-",
}

PROVINCIAS = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}



PERSONAL = {    
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

SCANNER = {
    "DENUNCIA": 4,
    "CONTROL PREVENTIVO": 6,
    "PATIO VALIJAS": 6,
    "ALLANAMIENTO" :8,
}

# Listas de prefijos conocidos
PREFIJOS = ["DOL", "OSR", "OSC", "OSL", "AP","FCB","RL","AJ","PS", 
            "CPE", "CSJ","CCC","CPF", "CPN","FBB", "FCR", "FCB",
            "FCT","FGR","FLP", "FMP","FMZ","FPO","FPA","FRE","FCA","FRO",
            "FSM","FTU", "CUIJ", "COIRON"]

# Unir las ubicaciones conocidas del diccionario `UNIDADES_MUNICIPIOS` con las anteriores
UBICACIONES = [
    "EZE", "AER", "SFO", "BHI", "MDP", "MDZ", "COR", "JUA", "LUI", "MLG", "RAF", 
    "RCU", "SRO", "JUJ", "SAL", "CAT", "LAR", "SGO", "TRH", "TUC", "IGU", "FSA",
    "ROS", "RCQ", "POS", "CRR", "LIB", "PAR", "RES", "SVO", "BAR", "TRE", "NEU",
    "CAL", "CHP", "CRV", "PMY", "ESQ", "GAL", "GDE", "USU", "VIE", "RG4",
    "UR1", "UR2", "UR3", "UR4", "UR5", "RG1", "RG2", "RG3", "RG5", "TO1", "TO2"
]

In [23]:
def filas_diferentes(row, df_base_informada):
    # Buscar si hay alguna fila en df_base_informada que sea idéntica en las columnas especificadas
    match = df_base_informada[
        (df_base_informada['ID_OPERATIVO'] != row['ID_OPERATIVO']) &
        (df_base_informada['FECHA'] != row['FECHA']) &
        (df_base_informada['LATITUD'] != row['LATITUD']) &
        (df_base_informada['LONGITUD'] != row['LONGITUD'])
    ]
    # Si no se encuentra un match, significa que la fila es diferente
    return match.empty


def procesar_unidad (row):
    unidad = row['UNIDAD_INTERVINIENTE']
    unidad ="UR1" if unidad in "DROPA I" else unidad
    return  unidad


def colocar_guion_espacio(texto):
    # Limpieza inicial del texto: eliminar caracteres innecesarios y normalizar guiones
    print("=== Inicio ===")
    print(f"Texto original: {texto}")
    
    caracteres_no_deseados = ['N°', ' N', '.', ':', '-', '"', '_', '´',"`"]

    # Eliminamos caracteres listados arriba en una sola expresión regular
    texto = re.sub(r'({})'.format('|'.join(map(re.escape, caracteres_no_deseados))), '', texto)
    
    # Reemplazar múltiples espacios por un solo guion para normalizar los separadores
    texto = re.sub(r'\s+', '-', texto)
    
    print(f"Texto después de la limpieza inicial: {texto}")

    # Inicializar componentes vacíos
    prefijo = ""
    numero = ""
    ubicacion = ""
    year = ""
    suffix = "-(1)"  # Valor por defecto del sufijo

    # Extraer prefijo si está en la lista de prefijos conocidos
    for p in PREFIJOS:
        if texto.upper().startswith(p):
            prefijo = p
            texto = texto[len(p):].strip()  # Recortar el prefijo del texto
            break
    print(f"Prefijo encontrado: {prefijo}")


    # Limpiar posibles espacios adicionales antes de buscar la ubicación
    texto = texto.strip()

    # Extraer ubicación si está en la lista de ubicaciones conocidas
    for u in UBICACIONES:
        if texto.upper().startswith(u):
            ubicacion = u
            texto = texto[len(u):].strip()  # Recortar la ubicación del texto
            break

    # Mejorar el chequeo para la ubicación, verificando si existe la ubicación en cualquier parte
    if not ubicacion:
        for u in UBICACIONES:
            if u in texto.upper():
                ubicacion = u
                texto = texto.replace(u, "", 1).strip()
                break
    print(f"Ubicación encontrada: {ubicacion}")

    # Extraer número principal
    match_numero = re.search(r"(\d+)", texto)
    if match_numero:
        numero = match_numero.group(0).zfill(4)
        texto = texto[len(match_numero.group(0)):].strip()  # Recortar el número del texto
    print(f"Número principal encontrado: {numero}")
    
    # Extraer año en formato de 4 dígitos
    match_year = re.search(r"(\d{4})", texto)
    if match_year:
        year = match_year.group(0)
        texto = texto[len(year):].strip()  # Recortar el año del texto
    print(f"Año encontrado: {year}")

    # Extraer sufijo al final (número entre paréntesis)
    match_suffix = re.search(r'\((\d+)\)$', texto)
    if match_suffix:
        suffix = f"-({match_suffix.group(1)})"
    print(f"Sufijo encontrado: {suffix}")

    # Formatear y retornar el texto resultante
    resultado = f"{prefijo}-{numero}-{ubicacion}/{year}{suffix}"
    print(f"Resultado formateado: {resultado}")
    print("=== Fin del proceso ===")
    return resultado

def formatear_contador(texto):
    texto_procesado = re.sub(r'-+\(\d+\)$', '', texto)
    return texto_procesado


def colocar_contador (df_operaciones, base):
    conteo_base_datos = base['ID_OPERATIVO'].value_counts()
    conteo_acumulado  = conteo_base_datos.to_dict()
    df_ordenes_no_informadas = pd.DataFrame()
    for index, row in df_operaciones.iterrows():
        id_operativa = row['ID_OPERATIVO']
        
        # Verificar cuántas veces ha aparecido el ID_operativa en total hasta ahora (base + nuevos)
        if id_operativa in conteo_acumulado:
            conteo_acumulado[id_operativa] += 1
        else:
            conteo_acumulado[id_operativa] = 1
        
        nuevo_id_procedimiento = f"{id_operativa}-({conteo_acumulado[id_operativa]})"
        
        df_ordenes_no_informadas.at[index, 'ID_PROCEDIMIENTO'] = nuevo_id_procedimiento
        
    return df_ordenes_no_informadas


def generar_uid_operaciones(row):
    texto = str(row['ID_OPERATIVO'])
    prefijo = ""
    for p in PREFIJOS:
        if texto.upper().startswith(p):
            prefijo = p
            texto = texto[len(p):].strip()  # Recortar el prefijo del texto
            break
    unidad = str(row['UNIDAD_INTERVINIENTE'])
    fecha_completa = str(row['FECHA'])
    fecha = fecha_completa.split()[0]  # Tomar solo la parte antes del espacio (la fecha)
    hora = str(row['HORA']).replace(":","-")
    conjunto = prefijo + "-" + unidad + "-" + fecha + "-" + hora
    
    return conjunto

def leer_excel_a_df(worksheet):
    titulos = [worksheet.cell(row=3, column=col).value for col in range(2, worksheet.max_column + 1)]

    data = []
    for row in worksheet.iter_rows(min_row=4, min_col=2, max_col=worksheet.max_column, values_only=True):
        data.append(row)

    df = pd.DataFrame(data, columns=titulos)
    return df


* Filtro cargo lo ya informado de operaciones y lo que se informo lo concateno  a lo de operaciones de 2024

In [24]:
base_2023_file = load_workbook("data/PLANILLA PSA CARGA DE DATOS - 2023 (OPERACIONES POLICIALES).xlsx")['GEOG. PROCEDIMIENTO']
base_opera_proce= leer_excel_a_df( base_2023_file )
base_opera_2023 = base_opera_proce [base_opera_proce['ID_PROCEDIMIENTO'].str.contains('OSL|OSR|OSC|DOL', na=False)]

print(f"FILTRADO': {base_opera_2023['ID_PROCEDIMIENTO'].count()}")


FILTRADO': 947


* Cargo lo informado de operaciones de 202 lo proceso y lo concate a lo que esta ya informado 

In [25]:
base_2024_file = load_workbook("data/BASE_JORGE.xlsx")['GEOG. PROCEDIMIENTO']
base_opera_proce =  leer_excel_a_df( base_2024_file )

base_opera_2024 = base_opera_proce [base_opera_proce['ID_PROCEDIMIENTO'].str.contains('OSL|OSR|OSC|DOL', na=False)]
print(f"FILTRADO': {base_opera_2024['ID_PROCEDIMIENTO'].count()}")


FILTRADO': 1233


*  Concateno las bases de 2023 y 2024 de operaciones y trato de estandarizar el formato 

In [27]:
base_operaciones = pd.concat([base_opera_2023 , base_opera_2024.iloc[1:]], ignore_index=True)

base_operaciones["ID_PROCEDIMIENTO"] = base_operaciones["ID_PROCEDIMIENTO"].apply(colocar_guion_espacio)
base_operaciones["ID_OPERATIVO"] = base_operaciones["ID_PROCEDIMIENTO"].apply(formatear_contador)
base_operaciones["UID"] = base_operaciones.apply(generar_uid_operaciones , axis=1)

base_operaciones.to_excel("data/Base_operaciones_2023_2024.xlsx", index=False)


=== Inicio ===
Texto original: OSR0007-UR1/2023
Texto después de la limpieza inicial: OSR0007UR1/2023
Prefijo encontrado: OSR
Ubicación encontrada: UR1
Número principal encontrado: 0007
Año encontrado: 2023
Sufijo encontrado: -(1)
Resultado formateado: OSR-0007-UR1/2023-(1)
=== Fin del proceso ===
=== Inicio ===
Texto original: OSR0007-UR1/2023(01)
Texto después de la limpieza inicial: OSR0007UR1/2023(01)
Prefijo encontrado: OSR
Ubicación encontrada: UR1
Número principal encontrado: 0007
Año encontrado: 2023
Sufijo encontrado: -(01)
Resultado formateado: OSR-0007-UR1/2023-(01)
=== Fin del proceso ===
=== Inicio ===
Texto original: OSR0007-UR1/2023(02)
Texto después de la limpieza inicial: OSR0007UR1/2023(02)
Prefijo encontrado: OSR
Ubicación encontrada: UR1
Número principal encontrado: 0007
Año encontrado: 2023
Sufijo encontrado: -(02)
Resultado formateado: OSR-0007-UR1/2023-(02)
=== Fin del proceso ===
=== Inicio ===
Texto original: OSR0007-UR1/2023(03)
Texto después de la limpieza in

# CARGA DE NUEVA PLANILLA SEMANAL

